# Quick Start

在这部分中，我们提供了一些基本但全面的示例，展示了Torch-Pruning的功能。

In [4]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.path.abspath("../"))

import oneflow as torch
from flowvision.models import resnet18
import oneflow_pruning as tp

### 方法1. 依赖图剪枝

``DependencyGraph`` : 是Torch-Pruning的基石，它自动识别和分组所有具有相互依赖关系的层。在结构剪枝中，具有依赖关系的两个层应同时进行剪枝。因此，要剪枝一个复杂的模型，我们需要仔细处理这些层。下面的示例显示了在ResNet-18中剪枝单个层的流程。

In [6]:
# 0. 准备模型和示例输入
model = resnet18(pretrained=True).eval()
example_inputs = torch.randn(1,3,224,224)

# 1. 为resnet18构建依赖图
DG = tp.DependencyGraph().build_dependency(model, example_inputs=example_inputs)

# 2. 选择要修剪的一些通道。这里我们修剪索引为[2, 6, 9]的通道。
pruning_idxs = pruning_idxs=[2, 6, 9]
pruning_group = DG.get_pruning_group( model.conv1, tp.prune_conv_out_channels, idxs=pruning_idxs )

# 3. 修剪与model.conv1耦合的所有分组层
if DG.check_pruning_group(pruning_group):
    pruning_group.prune()

KeyError: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

在调用``.exec``方法后，将对模型进行原地剪枝。打印模型后，我们可以注意到多个层，例如“model.conv1”，“model.bn1”和“model.layer1[0].conv1”都被Torch-Pruning剪枝了。

In [ ]:
print("After pruning:")
print(model)

让我们检查剪枝组。结果将显示剪枝操作如何触发（=>）另一个操作。

In [ ]:
print(pruning_group)

您还可以使用“get_all_groups”方法从“DependencyGraph”中获取所有组。

In [ ]:
all_groups = list(DG.get_all_groups())
print("Number of Groups: %d"%len(all_groups))
print("The last Group:", all_groups[-1])

---

### 2. 高级剪枝器

使用“DependencyGraph”对神经网络进行剪枝仍然可能很复杂，特别是对于具有众多层的模型。

因此，我们还提供了高级剪枝器来简化此过程。

例如，您可以使用简单的基于幅度的剪枝器轻松地剪枝ResNet18模型。

该方法删除网络中幅度较小的权重，从而得到一个更小、更快的模型，而准确性损失不会太大。


In [2]:
model = resnet18(pretrained=True)
example_inputs = torch.randn(1, 3, 224, 224)

# 0. importance criterion for parameter selections
# 定义参数选择的重要性标准
imp = tp.importance.MagnitudeImportance(p=2, group_reduction='mean')

# 1. ignore some layers that should not be pruned, e.g., the final classifier layer.
ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 1000:
        ignored_layers.append(m) # DO NOT prune the final classifier!

        
# 2. Pruner initialization
# 剪枝器初始化
iterative_steps = 5 # You can prune your model to the target sparsity iteratively.
pruner = tp.pruner.MagnitudePruner(
    model, 
    example_inputs, 
    global_pruning=False, # If False, a uniform sparsity will be assigned to different layers.
    importance=imp, # importance criterion for parameter selection
    iterative_steps=iterative_steps, # the number of iterations to achieve target sparsity
    ch_sparsity=0.5, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    ignored_layers=ignored_layers,
)
# 计算模型的基本计算量和参数数量
base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)
for i in range(iterative_steps):
    # 3. the pruner.step will remove some channels from the model with least importance
    # pruner.step 将从具有最小重要性的模型中删除一些通道
    pruner.step()
    
    # 4. Do whatever you like here, such as fintuning
    # 在此处进行微调等操作
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    print(model)
    print(model(example_inputs).shape)
    print(
        "  Iter %d/%d, Params: %.2f M => %.2f M"
        % (i+1, iterative_steps, base_nparams / 1e6, nparams / 1e6)
    )
    print(
        "  Iter %d/%d, MACs: %.2f G => %.2f G"
        % (i+1, iterative_steps, base_macs / 1e9, macs / 1e9)
    )
    # finetune your model here
    # finetune(model)
    # ...



NameError: name 'resnet18' is not defined